# 4.3. Training Jobs and Sagemaker (tg.common.delivery.sagemaker)

## Preparing the training task and data

Another scenario for delivery is `Sagemaker` training, that is applicable to the descendants of `TrainingTask`. We will demonstrate it with `SingleFrameTrainingTask`, as it has simpler setup, and titanic dataset. 

First, we need to create a dataset and place it in the right folder.

In [1]:
from sklearn import datasets
import pandas as pd
from pathlib import Path
import os

df = pd.read_csv('titanic.csv')
df = df.set_index('PassengerId')
for c in ['Pclass','SibSp','Parch','Survived']:
    df[c] = df[c].astype(float)
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
df = df[features+['Survived']]
datasets_folder = Path('temp/datasets/titanic')
dataset_file = datasets_folder/'titanic_project/titanic.parquet'
os.makedirs(dataset_file.parent, exist_ok=True)
df.to_parquet(dataset_file)

We will store it locally. We will not actually run this task on the `Sagemaker`, hence, there is no need to upload it. In real setup, you would need to upload the dataset to your `[bucket]`, respecting the following convention:

* Datasets are uploaded to `[bucket]/sagemaker/[project_name]/datasets/`
* Output of the training jobs is placed to `[bucket]/sagemaker/[project_name]/output`

In [2]:
from tg.common.ml import single_frame_training as sft
from tg.common.ml import dft
from sklearn.metrics import roc_auc_score

task = sft.SingleFrameTrainingTask(
    data_loader = sft.DataFrameLoader('Survived'),
    model_provider=sft.ModelProvider(sft.ModelConstructor(
            'sklearn.linear_model:LogisticRegression',
            max_iter = 1000),
        transformer = dft.DataFrameTransformerFactory.default_factory(),
        keep_column_names=False),
    evaluator=sft.Evaluation.binary_classification,
    splitter=sft.FoldSplitter(),
    metrics_pool = sft.MetricPool().add_sklearn(roc_auc_score)        
    )

To start Sagemaker training even on the local machine, one needs `AWS_ROLE`. We will import it from `environment.env` file:

In [3]:
from tg.common import Loc
import dotenv

dotenv.load_dotenv(Loc.root_path/'environment.env')
'AWS_ROLE' in os.environ

True

Sagemaker delivery has a similar structure to the SSH/Docker: SagemakerOptions, SagemakerConfig, and three executors. As with SSH/Docker, the best way to use all this is to write a SagemakerRoutine which will set up all these.

Some notes before we start:
  * The task is not, by itself, a job, it is not self-contained, as the artefacts output is controlled by `TrainingEnvironment`. So, `SagemakerJob` is a job in the sence of `tg.common.delivery` that wraps the task and adopts its behaviour to sagemaker. Other cloud providers will probably required different tasks.
  * Sagemaker itself reqiures some specifics in container files, so this also needs to be reflected.
  * There are many dependencies required for training are, so we will need to change the default dependency lists.



In [4]:
from tg.common.delivery.sagemaker import (SagemakerJob, SagemakerAttachedExecutor, SagemakerLocalExecutor, 
                                          DOCKERFILE_TEMPLATE, SagemakerOptions, SagemakerConfig)
from tg.common.delivery.delivery import Packaging, Containering, DependencyList
from yo_fluq_ds import *

dependencies = FileIO.read_json('dependencies.json')
dependencies = DependencyList('training', dependencies)


class SagemakerRoutine:
    def __init__(self,
                 task,
                 dataset: str,
                 project_name: str,
                 ):
        name = type(task).__name__
        task.info['name'] = name
        version = '0'
        job = SagemakerJob(task)
        packaging = Packaging(name, version, dict(job=job))
        packaging.dependencies = [dependencies]
        packaging.silent = True

        containering = Containering.from_packaging(packaging)
        containering.dependencies = [dependencies]
        containering.dockerfile_template = DOCKERFILE_TEMPLATE
        containering.run_file_name='train.py'
        containering.silent = True

        settings = SagemakerOptions(
            os.environ.get('AWS_ROLE'),
            None,
            project_name,
            datasets_folder,
            dataset,
        )

        self.config = SagemakerConfig(
            job,
            packaging,
            containering,
            settings
        )
        
    def attached(self):
        return SagemakerAttachedExecutor(self.config)
        
    def local(self):
        return SagemakerLocalExecutor(self.config)
    
routine = SagemakerRoutine(task,'titanic.parquet','titanic_project')
result = routine.attached().execute()
result['metrics']

  0%|          | 0/1 [00:00<?, ?it/s]

2022-12-20 17:37:25.113050 INFO: Starting stage 1/1
2022-12-20 17:37:25.233140 INFO: Completed stage 1/1
2022-12-20 17:37:25.236670 INFO: ###roc_auc_score_test:0.8538095238095237
2022-12-20 17:37:25.237406 INFO: ###roc_auc_score_train:0.8600247283139194


{'roc_auc_score_test': 0.8538095238095237,
 'roc_auc_score_train': 0.8600247283139194}

Now we will run it in the local container:

In [5]:
id = routine.local().execute()

sha256:c5e1a98741882ce94e09ce458d7ace052fbdd2517633901743fe506d457c8cfd
Creating zuve7o5y6g-algo-1-zipal ... 
Attaching to zuve7o5y6g-algo-1-zipal2mdone
zuve7o5y6g-algo-1-zipal | 2022-12-20 16:37:41,030 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
zuve7o5y6g-algo-1-zipal | 2022-12-20 16:37:41,064 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
zuve7o5y6g-algo-1-zipal | 2022-12-20 16:37:41,084 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
zuve7o5y6g-algo-1-zipal | 2022-12-20 16:37:41,097 sagemaker-containers INFO     Invoking user script
zuve7o5y6g-algo-1-zipal | 
zuve7o5y6g-algo-1-zipal | Training Env:
zuve7o5y6g-algo-1-zipal | 
zuve7o5y6g-algo-1-zipal | {
zuve7o5y6g-algo-1-zipal |     "additional_framework_parameters": {},
zuve7o5y6g-algo-1-zipal |     "channel_input_dirs": {
zuve7o5y6g-algo-1-zipal |         "training": "/opt/ml/input/data/training"
zuve7o5y6g-algo-1-zipal |     },
zuve7o5y6g-

zuve7o5y6g-algo-1-zipal | 2022-12-20 16:37:44,919 sagemaker-containers INFO     Reporting training SUCCESS
zuve7o5y6g-algo-1-zipal exited with code 0
Aborting on container exit...
===== Job Complete =====


The result is stored in the local file system in the same format it would be stored in S3. This is a zipped file that contains not only the output, but also the package information:

In [6]:
loader = routine.local().load_result(id)

package.tag.gz
task.json
package.json
runs/
runs/0/
runs/0/info.pkl
runs/0/metrics.pkl
runs/0/model.pkl
runs/0/result_df.parquet
runs/0/test_splits.pkl
runs/0/train_split.pkl
runs/0/training_task.pkl


We can now read the dataframe with the results:

In [7]:
df = pd.read_parquet(loader.get_path('runs/0/result_df.parquet'))

We can also read pickled objects, although non-directly:

In [8]:
import traceback
try:
    FileIO.read_pickle(loader.get_path('runs/0/training_task.pkl'))
except:
    print(traceback.format_exc())

Traceback (most recent call last):
  File "/tmp/ipykernel_3186/1779605262.py", line 3, in <module>
    FileIO.read_pickle(loader.get_path('runs/0/training_task.pkl'))
  File "/home/yura/anaconda3/envs/fol/lib/python3.8/site-packages/yo_fluq_ds/_misc/io.py", line 17, in read_pickle
    return pickle.load(file)
ModuleNotFoundError: No module named 'SingleFrameTrainingTask__0'



This is due to the fact that the delivered training task was delivered, and the delivery process changed the module name. But the `loader` contains the method to unpickle such files regardless:

In [9]:
loader.unpickle('runs/0/training_task.pkl')

## Automatic task name's assignment

When multiple tasks are running, it's quite handy to assign to each a name that would represent the parameters of the task. Out initial idea was to implement this logic inside the task, but the downside of this approach is that parameters are many, while length of the task's name in Sagemaker is limited, and quickly reached. 

The alternative solution is to, first, use a factory method that builds tasks:

In [10]:
from yo_fluq_ds import Obj

def build(
    learning_rate=1, 
    network_size=[10,10], 
    context_length = 10,
):
    return Obj(info=dict(name=''))

This `build` method returns a mock for training task: we are now interested only in `info` field of the task, that will contain the name.

In [11]:
from tg.common.delivery.sagemaker import Autonamer

Autonamer(build).build_tasks(learning_rate = [1, 2], network_size = [[10], [10, 5]])

[{'info': {'name': 'LR1-NS10'}},
 {'info': {'name': 'LR1-NS10-5'}},
 {'info': {'name': 'LR2-NS10'}},
 {'info': {'name': 'LR2-NS10-5'}}]

As we can see, `Autonamer` will instantiate all the tasks and assign automatically generated names to them. Note that it does not create entry for `context_length` in the name, as it is not variable in this run. 

